In [2]:
# Installing Interactive Brokers API

# This is not installed with pip, instead you must install TWS or IB Gateway 
# then install the TWS API and install via the following commands
# `python -m pip install setuptools`
# `cd C:\TWS API\source\pythonclient\`
# `python setup.py install`
import ibapi

In [29]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
import threading
import time

class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []

    def historicalData(self, reqId, bar):
        self.data.append([bar.date, bar.close, bar.open, bar.volume, bar.high, bar.low])

def run_loop():
    app.run()

# Establish connection to IB
app = IBApi()
app.connect("127.0.0.1", 7497, clientId=123)
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(10)  # Ensure that the connection is established

# Define the contract
contract = Contract()
contract.symbol = "SPY"
contract.secType = "STK"
contract.exchange = "SMART"
contract.currency = "USD"

# Request historical data
app.reqHistoricalData(reqId=1, contract=contract,
                      endDateTime='', durationStr='1 Y',
                      barSizeSetting='1 day', whatToShow='TRADES',
                      useRTH=True, formatDate=1, keepUpToDate=False,
                      chartOptions=[])

time.sleep(5)  # Wait for data to be fetched

# Disconnect
app.disconnect()

# Create DataFrame
df = pd.DataFrame(app.data, columns=['Date', 'Close', 'Open', 'Volume', 'High', 'Low'])
print(df)


ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


         Date   Close    Open    Volume    High     Low
0    20221228  376.66  381.34  45531220  383.39  376.42
1    20221229  383.44  379.66  43269617  384.35  379.08
2    20221230  382.43  380.64  50712172  382.53  378.43
3    20230103  380.82  384.42  49025780  386.43  377.83
4    20230104  383.76  383.18  59492813  385.88  380.00
..        ...     ...     ...       ...     ...     ...
246  20231220  468.26  473.96  67376027  475.90  467.82
247  20231221  472.70  471.32  57760976  472.98  468.84
248  20231222  473.65  473.86  44344028  475.38  471.70
249  20231226  475.65  474.08  31409570  476.58  473.99
250  20231227  476.31  475.44  19242476  476.66  475.08

[251 rows x 6 columns]
